## Diagnóstico y Transformación de Datos Crudos

**Contexto:**  
Los modelos de *Machine Learning* solo funcionan correctamente si los datos han sido preparados de forma adecuada.  
Esta actividad tiene como objetivo aplicar un diagnóstico y tratamiento básico sobre un *dataset* realista, para asegurar que esté listo para el modelado.

**Consigna:**  
Trabaja sobre un conjunto de datos crudos que contenga problemas típicos:
- Valores nulos.
- Variables categóricas.
- Escalas desiguales.
- Posibles duplicados.

Identifica y aplica las transformaciones necesarias para dejarlo preparado para el entrenamiento de un modelo de *Machine Learning*.

---

### Paso a Paso

1. **Carga de datos:**  
   Importa el *dataset* simulado (provisto por el docente).

2. **Diagnóstico inicial:**  
   - Detecta valores faltantes.  
   - Identifica filas o registros duplicados.  
   - Reconoce variables categóricas.  
   - Verifica la existencia de escalas diferentes entre variables numéricas.

3. **Tratamiento de datos:**  
   - **Imputación:** utilizar `SimpleImputer` con media o mediana, según corresponda.  
   - **Codificación:** aplicar `OneHotEncoder` o `LabelEncoder` para variables categóricas.  
   - **Escalamiento:** usar `MinMaxScaler` o `StandardScaler` para uniformar las escalas.

4. **Comparación y reflexión:**  
   - Mostrar el estado de los datos antes y después del tratamiento.  
   - Explicar los cambios realizados y cómo mejoran la preparación de los datos para el modelado.


In [ ]:
from google.colab import files
import pandas as pd
import io

uploaded = files.upload()
filename = next(iter(uploaded))

df = pd.read_csv(io.BytesIO(uploaded[filename]), sep=';', encoding='latin1')

print(f"Archivo cargado: {filename}")
print(df.head())



Saving Dataset_Vivero.csv to Dataset_Vivero (10).csv
Archivo cargado: Dataset_Vivero (10).csv
   PlantaID      Tipo Altura_cm Ancho_cm Precio  En_stock
0         1      Rosa      18,3     42,1    4,7       0.0
1         2       NaN      81,8     14,2   20,0       NaN
2         3   Tulipán      48,0     18,1   20,3       1.0
3         4   Tulipán      68,4     54,9   23,4       1.0
4         5  Orquídea     110,3     40,3   26,2       0.0


In [ ]:
df.head()

,PlantaID,Tipo,Altura_cm,Ancho_cm,Precio,En_stock
0,1,Rosa,"18,3","42,1","4,7",0.0
1,2,NaN,"81,8","14,2","20,0",NaN
2,3,Tulipán,"48,0","18,1","20,3",1.0
3,4,Tulipán,"68,4","54,9","23,4",1.0
4,5,Orquídea,"110,3","40,3","26,2",0.0


# Se logra identificar que hay datos NaN los cuales se tienen que trabajar

In [ ]:
# Info general y tipos
print(df.info())

# Valores faltantes por columna
print(df.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   PlantaID   102 non-null    int64  
 1   Tipo       97 non-null     object 
 2   Altura_cm  97 non-null     object 
 3   Ancho_cm   102 non-null    object 
 4   Precio     98 non-null     object 
 5   En_stock   85 non-null     float64
dtypes: float64(1), int64(1), object(4)
memory usage: 4.9+ KB
None
PlantaID      0
Tipo          5
Altura_cm     5
Ancho_cm      0
Precio        4
En_stock     17
dtype: int64


Se identifica que algunas columnas como Altura_cm,Ancho_cm, Precio, se encuentran en tipo objeto, lo cual es un gran problema para el análisis, por ende se necesita cambiar.

Otro dato importante es que hay varios nulos los cuales se tienen que trabajar

In [ ]:
cols_to_convert = ['Altura_cm', 'Ancho_cm', 'Precio']

for col in cols_to_convert:
    df[col] = df[col].astype(str).str.replace(',', '.')
    df[col] = pd.to_numeric(df[col], errors='coerce')

print("Tipos tras conversión:")
print(df.dtypes)


Tipos tras conversión:
PlantaID       int64
Tipo          object
Altura_cm    float64
Ancho_cm     float64
Precio       float64
En_stock     float64
dtype: object


In [ ]:
from sklearn.impute import SimpleImputer

# Columnas numéricas a imputar
num_cols = ['Altura_cm', 'Ancho_cm', 'Precio', 'En_stock']
imputer_num = SimpleImputer(strategy='median')
df[num_cols] = imputer_num.fit_transform(df[num_cols])

# Columna categórica a imputar
cat_col = ['Tipo']
imputer_cat = SimpleImputer(strategy='most_frequent')
df[cat_col] = imputer_cat.fit_transform(df[cat_col])

print("Valores faltantes después de imputación:")
print(df.isnull().sum())


Valores faltantes después de imputación:
PlantaID     0
Tipo         0
Altura_cm    0
Ancho_cm     0
Precio       0
En_stock     0
dtype: int64


En este caso se decidió de imputar las columnas numericas con la media de cada una, me parece que cuando hablamos de flores, la media nos puede ayudar al proceso de trabajo.

En las columnas categóricas, decidí colocar como lo mas frecuente, para asi poder trabajarlo, pero lo mas común es buscar el origen de los datos para intentar completar esos datos vacios.

In [ ]:
print(f"Filas duplicadas: {df.duplicated().sum()}")
df = df.drop_duplicates()
print(f"Filas después de eliminar duplicados: {len(df)}")


Filas duplicadas: 2
Filas después de eliminar duplicados: 100


## En este caso se decide eliminar las filas duplicadas

In [ ]:
# Usando pandas get_dummies para One-Hot Encoding
df_encoded = pd.get_dummies(df, columns=['Tipo'], prefix='Tipo')

print(df_encoded.head())


   PlantaID  Altura_cm  Ancho_cm  Precio  En_stock  Tipo_Girasol  \
0         1       18.3      42.1     4.7       0.0         False   
1         2       81.8      14.2    20.0       0.0         False   
2         3       48.0      18.1    20.3       1.0         False   
3         4       68.4      54.9    23.4       1.0         False   
4         5      110.3      40.3    26.2       0.0         False   

   Tipo_Orquídea  Tipo_Rosa  Tipo_Tulipán  
0          False       True         False  
1           True      False         False  
2          False      False          True  
3          False      False          True  
4           True      False         False  


En este paso se aplicó One-Hot Encoding con pandas.get_dummies para transformar la variable categórica Tipo en columnas binarias independientes.
Esto permite que el modelo de Machine Learning procese correctamente la información categórica.
Cada nueva columna indica con valores booleanos la presencia o ausencia de una categoría específica.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Columnas numéricas a escalar
cols_num = ['Altura_cm', 'Ancho_cm', 'Precio', 'En_stock']

scaler = StandardScaler()

# Ajustar y transformar
df[cols_num] = scaler.fit_transform(df[cols_num])

# Mostrar resultado
print(df.head())
print(df[cols_num].describe())


   PlantaID      Tipo  Altura_cm  Ancho_cm    Precio  En_stock
0         1      Rosa  -1.610158  0.425952 -1.537861 -0.833616
1         2  Orquídea   0.487563 -1.485899  0.128518 -0.833616
2         3   Tulipán  -0.629019 -1.218651  0.161192  1.199593
3         4   Tulipán   0.044895  1.303074  0.498825  1.199593
4         5  Orquídea   1.429060  0.302607  0.803783 -0.833616
          Altura_cm      Ancho_cm        Precio      En_stock
count  1.000000e+02  1.000000e+02  1.000000e+02  1.000000e+02
mean   5.895284e-16 -5.195844e-16 -3.774758e-17  8.215650e-17
std    1.005038e+00  1.005038e+00  1.005038e+00  1.005038e+00
min   -1.696049e+00 -1.753148e+00 -1.668557e+00 -8.336158e-01
25%   -8.800850e-01 -8.263448e-01 -7.945251e-01 -8.336158e-01
50%    4.489454e-02  1.518517e-01  1.012897e-01 -8.336158e-01
75%    9.062817e-01  8.028407e-01  8.119513e-01  1.199593e+00
max    1.699947e+00  1.618290e+00  1.729549e+00  1.199593e+00


Se aplicó escalamiento estándar (StandardScaler) a las variables numéricas para normalizar su distribución.
Este proceso ajusta los datos para que tengan media 0 y desviación estándar 1, evitando que las diferencias de escala influyan en el modelo.
Como resultado, todas las variables numéricas quedaron en la misma magnitud y son comparables entre sí.


#Conclusión

Con el diagnóstico y tratamiento aplicados, el conjunto de datos quedó libre de valores faltantes, correctamente codificado y con las variables numéricas escaladas.
Esto garantiza que todas las características estén en un formato adecuado y comparable, optimizando el rendimiento y la precisión de futuros modelos de Machine Learning.